In [1]:
!pip install alibi-detect

  Using cached alibi_detect-0.12.0-py3-none-any.whl.metadata (28 kB)
Using cached alibi_detect-0.12.0-py3-none-any.whl (381 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 1.1 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 687.1 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 1.0 MB/s eta 0:00:0000:010m00:01


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier
from sklearn import metrics
from sklearn.pipeline import make_pipeline

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import alibi
from alibi_detect.cd import ChiSquareDrift, TabularDrift
from alibi_detect.utils.saving import save_detector, load_detector


Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'alibi_detect'

In [ ]:
# Grab the data
wine_data = load_wine()
feature_names = wine_data.feature_names
X, y = wine_data.data, wine_data.target 

# Make a 50/50 reference/test split
X_ref, X_test, y_ref, y_test = train_test_split(X, y,
                                                test_size=0.50,
                                                random_state=42)

In [ ]:
# Initialise the detector
cd = TabularDrift(p_val=.05, X_ref=X_ref)

In [ ]:
# Check for drift 
preds = cd.predict(X_test)
labels = ['No', 'Yes']
print('Drift: {}'.format(labels[preds['data']['is_drift']]))

In [ ]:
# Check for drift - X_test_cal_error is simulated calibration error test set
X_test_cal_error = 1.1*X_test
preds = cd.predict(X_test_cal_error)
labels = ['No', 'Yes']
print('Drift: {}'.format(labels[preds['data']['is_drift']]))

In [ ]:
# check for drift at the level of features
fpreds = cd.predict(X_test+4, drift_type='feature')

In [ ]:
results = []
for f in range(cd.n_features):
    
    stat = 'K-S' #all numeric features for this dataset
    fname = feature_names[f]
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    
    results.append(
        {
            'feature': fname,
            'statistic': 'K-S',
            'statisticValue': float(stat_val),
            'driftResult': labels[is_drift],
            'pValue': float(p_val)
        }
    )

In [ ]:
print(json.dumps(results, indent=4, sort_keys=True))